In [15]:
import logging
from mne_bids.utils import _write_json
import os
logging.basicConfig(format='%(asctime)s : %(levelname)s: %(message)s', level=logging.DEBUG)


class Block:
    def __init__(self, a, b, c, d):
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        self.quality_scores = {
            'overall_high_amp': a,
            'times_high_var': b,
            'ratio_bad_chans': c,
            'chan_high_var': d
        }

    def preprocess(self):
        x = self.a
        results = {'preprocessed': x}
        return results


block_list = ['sub-1', 'sub-2', 'sub-3']
block_map = {}
block_map[block_list[0]] = Block(0.05, 0.05, 0.1, 0.1)
block_map[block_list[1]] = Block(0.15, 0.15, 0.2, 0.2)
block_map[block_list[2]] = Block(0.25, 0.25, 0.35, 0.35)
processed_list = block_list

def get_quality_ratings():
    blocks = [block_map[x] for x in processed_list]
    q_scores = [z.quality_scores for z in blocks]
    ratings = [rate_quality(w) for w in q_scores]
    return ratings

def save_project(name, data):
    result_file = os.path.sep + 'content' + os.path.sep + name + '_results.json'
    _write_json(result_file, data, overwrite=True, verbose=True)

# Function that rates EEG datasets depending on quality_metrics values and predefined rating values
def rate_quality(quality_metrics, overall_Good_Cutoff: float = 0.1, overall_Bad_Cutoff: float = 0.2,
               time_Good_Cutoff: float = 0.1, time_Bad_Cutoff: float = 0.2,
               bad_Channel_Good_Cutoff: float = 0.15, bad_Channel_Bad_Cutoff: float = 0.3, channel_Good_Cutoff: float = 0.15,
               channel_Bad_Cutoff: float = 0.3):
   """
   Rates datasets, based on quality measures calculated with calcQuality().
   The possible ratings:
       Good overall rating
       Regular overall rating
       Bad overall rating

   Parameters
   ----------
   quality_metrics : dict
                   a dictionary containing the quality metrics to rate the dataset.
   overall_Good_Cutoff : float
                   cutoff for "Good" quality based on  overall high amplitude data points [0.1].
   overall_Bad_Cutoff : float
                   cutoff for "Bad" quality based on overall high amplitude data point [0.2].
   time_Good_Cutoff : float
                   cutoff for "Good" quality based on time points of high variance across channels [0.1].
   time_Bad_Cutoff : float
                   cutoff for "Bad" quality based on time points of high variance across channels [0.2].
   bad_Channel_Good_Cutoff : float
                   cutoff for "Good" quality based on ratio of bad channels [0.15].
   bad_Channel_Bad_Cutoff : float
                   cutoff for "Bad" quality based on ratio of bad channels [0.3].
   channel_Good_Cutoff : float
                   cutoff for "Good" quality based on channels of high variance across time [0.15].
   channel_Bad_Cutoff : float
                   cutoff for "Bad" quality based on channels of high variance across time [0.3].
   Returns
   -------
   dataset_qualification : dict
                   a dictionary indicating is the dataset if "Good" = 100, "Regular" = 50 or "Bad" = 0.
   """

   # Check that the values in quality_metrics{} are positive numbers not equal to 0
   for i in quality_metrics.values():
       # Verify if any value in quality_metrics is a string
       if isinstance(i, str):
           logging.log(40, "Some value of Quality Metrics is not a positive number, please verify your EEG input data")
           break
       # Verify if any value in quality_metrics is bool type
       elif isinstance(i, bool):
           logging.log(40, "Some value of Quality Metrics is not a positive number, please verify your EEG input data")
           break
       # Verify if any value in quality_metrics is a negative number
       elif i < 0:
           logging.log(40, "Some value of Quality Metrics is not a positive number, please verify your EEG input data")
           break
       else:

           # Rating of EEG DATA according to the values of quality_metrics

           # The function rates the EEG DATA with the rule that the rating depends on the WORST rating

           if quality_metrics['overall_high_amp'] > overall_Bad_Cutoff or quality_metrics['times_high_var'] > time_Bad_Cutoff or quality_metrics['ratio_bad_chans'] > bad_Channel_Bad_Cutoff or quality_metrics['chan_high_var'] > channel_Bad_Cutoff:
               dataset_qualification = {'dataset_qualification': 0}  # Bad EEG dataset rating if any rating is BAD
               #logging.info("Bad dataset: %s", dataset_qualification['dataset_qualification'])
               return dataset_qualification
           elif quality_metrics['overall_high_amp'] < overall_Good_Cutoff and quality_metrics['times_high_var'] < time_Good_Cutoff and quality_metrics['ratio_bad_chans'] < bad_Channel_Good_Cutoff and quality_metrics['chan_high_var'] < channel_Good_Cutoff:
               dataset_qualification = {'dataset_qualification': 100}  # Good EEG dataset rating if all ratings are GOOD
               #logging.info("Good dataset: %s", dataset_qualification['dataset_qualification'])
               return dataset_qualification
           else:
               dataset_qualification = {'dataset_qualification': 50}  # Regular EEG dataset rating if any rating is REGULAR
               #logging.info("Regular dataset: %s", dataset_qualification['dataset_qualification'])
               return dataset_qualification


ratings = get_quality_ratings()
print(ratings)

data = {}
for i in range(len(ratings)):
    data[block_list[i]] = ratings[i]

save_project('Test', data) 

[{'dataset_qualification': 100}, {'dataset_qualification': 50}, {'dataset_qualification': 0}]

Writing '/content/Test_results.json'...

{
    "sub-1": {
        "dataset_qualification": 100
    },
    "sub-2": {
        "dataset_qualification": 50
    },
    "sub-3": {
        "dataset_qualification": 0
    }
}
